In [254]:
import nltk
from nltk.corpus import brown, words, stopwords, reuters, wordnet, abc, gutenberg, nps_chat
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('train_lower_nostopwords.csv')

In [255]:
nltk.download('nps_chat')

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\nps_chat.zip.


True

In [256]:
# create a list of correctly spelled words from various corpi
bw = brown.words()
ww = words.words()
rw = reuters.words()
wnw = wordnet.words()
aw = abc.words()
gw = gutenberg.words()
nw = nps_chat.words()
wordlist_lower = []

for i in bw:
    wordlist_lower.append(i.lower())
    
for i in ww:
    wordlist_lower.append(i.lower())
    
for i in rw:
    wordlist_lower.append(i.lower())
    
for i in wnw:
    wordlist_lower.append(i.lower())

for i in bw:
    wordlist_lower.append(i.lower())
    
for i in aw:
    wordlist_lower.append(i.lower())
    
for i in gw:
    wordlist_lower.append(i.lower())
    
for i in nw:
    wordlist_lower.append(i.lower())
    
test = ' '.join(wordlist_lower)
strip_test = re.sub(r"'s", r'', test)
strip_test = re.sub(r'([^a-zA-Z])', r' ', strip_test)
strip_test = re.sub(r'(\s{2,})', r' ', strip_test)
spell_words = strip_test.split(' ')

In [257]:
# create list of unique tokens for a corpus
def unique_words(tokens):
    r = []

    for x in tokens:
        if len(x)>2:
            r.append(x)
    r_array = np.array(r)
    words = np.unique(r_array)
    return list(words)

In [258]:
word_list = unique_words(spell_words)
word_list

['aaa',
 'aaaaaaaaaaaaaaaaa',
 'aaahhhh',
 'aaawww',
 'aab',
 'aabex',
 'aabye',
 'aac',
 'aachen',
 'aachener',
 'aacute',
 'aad',
 'aagiy',
 'aah',
 'aaica',
 'aaium',
 'aaix',
 'aal',
 'aaland',
 'aalborg',
 'aalge',
 'aalii',
 'aalst',
 'aalto',
 'aam',
 'aame',
 'aami',
 'aancor',
 'aani',
 'aao',
 'aap',
 'aar',
 'aardvark',
 'aardwolf',
 'aare',
 'aarhus',
 'aarn',
 'aarnoud',
 'aaron',
 'aaronic',
 'aaronical',
 'aaronite',
 'aaronites',
 'aaronitic',
 'aaroy',
 'aarp',
 'aart',
 'aaru',
 'aas',
 'aassi',
 'aati',
 'aave',
 'aba',
 'ababa',
 'ababdeh',
 'ababua',
 'abac',
 'abaca',
 'abacate',
 'abacay',
 'abacinate',
 'abacination',
 'abaciscus',
 'abacist',
 'aback',
 'abactinal',
 'abactinally',
 'abaction',
 'abactor',
 'abaculus',
 'abacus',
 'abadan',
 'abaddon',
 'abadite',
 'abaff',
 'abaft',
 'abagtha',
 'abaisance',
 'abaiser',
 'abaissed',
 'abal',
 'abalienate',
 'abalienation',
 'abalone',
 'abama',
 'abamp',
 'abampere',
 'abana',
 'abandon',
 'abandonable',
 'aba

In [295]:
df = pd.read_csv('train_update_nostopwords.csv')
token_list = list(df['tweet_tokens'])

tokes = ''
for sublist in token_list:
    for item in sublist:
        tokes += item
tokens = tokes.split(' ')

In [296]:
tokens

['our',
 'deeds',
 'reason',
 'earthquake',
 'may',
 'allah',
 'forgiveforest',
 'fire',
 'near',
 'ronge',
 'sask',
 'canadaall',
 'residents',
 'ask',
 'shelter',
 'place',
 'notify',
 'officer',
 'evacuation',
 'shelter',
 'place',
 'order',
 'expectpeople',
 'receive',
 'wildfires',
 'evacuation',
 'order',
 'californiajust',
 'get',
 'send',
 'photo',
 'ruby',
 'alaska',
 'smoke',
 'wildfires',
 'pour',
 'schoolrocky',
 'fire',
 'update',
 'california',
 'hwy',
 'close',
 'directions',
 'due',
 'lake',
 'county',
 'fire',
 'afire',
 'wildfiresflood',
 'disaster',
 'heavy',
 'rain',
 'cause',
 'flash',
 'flood',
 'streets',
 'manitou',
 'colorado',
 'springs',
 'areastop',
 'hill',
 'see',
 'fire',
 'woodsthere',
 'emergency',
 'evacuation',
 'happen',
 'build',
 'across',
 'streetafraid',
 'tornado',
 'come',
 'areathree',
 'people',
 'die',
 'heat',
 'wave',
 'farhaha',
 'south',
 'tampa',
 'get',
 'flood',
 'hah',
 'wait',
 'second',
 'live',
 'south',
 'tampa',
 'what',
 'gonna

In [297]:
toke_list = unique_words(tokens)

In [298]:
toke_list.sort() 
toke_list.sort(key=len, reverse=True) 
word_list.sort() 
word_list.sort(key=len, reverse=True) 

In [299]:
len(toke_list)

18197

In [282]:
misspelled = []
misspelled = [i for i in tokens if i not in word_list]

In [283]:
misspelled

['theresmorewherethatcamefromseanhannity',
 'measuresarrestpastorngangaaftershock',
 'bathandnortheastsomersetearthquake',
 'jakartapostdisasterlovelylikelaura',
 'spiderwebobliteratebreakingnewslh',
 'thisiswhywecanthavenicethingscake',
 'thisiswhywecanthavenicethingsdave',
 'cantwaittoplayinminneapolisweek',
 'growingupincoloradocacheadvance',
 'triciaoneillphotomyrtlegroggins',
 'beclearoncancerhighqualitybird',
 'changepilipinasrealdonaldtrump',
 'newbeginningsanimalrescueblood',
 'bangladeshfloodtonymcguinness',
 'blacklivesmatterfewmoretweets',
 'disclaimeraftershockkjfordays',
 'floodingfloodingadorableapple',
 'givebackkalinwhiteaccountluke',
 'jonathanferrellcharlesdagnall',
 'thatswhatfriendsareforfreeway',
 'antiochhickoryhollowtnlender',
 'intelligencebarjustintrudeau',
 'mypillowstudiostormjohngreen',
 'rezaphotographypolicyholders',
 'rowysolouisvillesteveycheese',
 'troubleonmymindjetixrestored',
 'ashestoashesengineermatarai',
 'bioterrorismdrrichardbesser',
 'conversat